<h1>Содержание <span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-библиотек" data-toc-modified-id="Подключение-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение библиотек</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Первичный-осмотр" data-toc-modified-id="Первичный-осмотр-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Первичный осмотр</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Деление-данные-на-выборки" data-toc-modified-id="Деление-данные-на-выборки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Деление данные на выборки</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Создание-признаков-для-обучения" data-toc-modified-id="Создание-признаков-для-обучения-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Создание признаков для обучения</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Простой-классификатор" data-toc-modified-id="Простой-классификатор-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Простой классификатор</a></span></li><li><span><a href="#Решающие-дерево" data-toc-modified-id="Решающие-дерево-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Решающие дерево</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#Подготовка-признаков-для-тестирования" data-toc-modified-id="Подготовка-признаков-для-тестирования-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка признаков для тестирования</a></span></li><li><span><a href="#Проверка-на-адекватность" data-toc-modified-id="Проверка-на-адекватность-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Проверка на адекватность</a></span><ul class="toc-item"><li><span><a href="#Простой-классификатор" data-toc-modified-id="Простой-классификатор-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Простой классификатор</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Задание 

Обучите модель классифицировать комментарии на позитивные и негативные.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Подключение библиотек 

In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')


import re 

import warnings
warnings.filterwarnings('ignore')


!pip3 install imblearn -U
from imblearn.over_sampling import SMOTE

from pymystem3 import Mystem

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, make_scorer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Requirement already up-to-date: imblearn in /home/alex/.local/lib/python3.8/site-packages (0.0)


## Подготовка

### Первичный осмотр

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


#### Вывод

Данные не содержат проблем 

### Деление данные на выборки

In [5]:
train, test = train_test_split(data, test_size=0.2, random_state=12345) 

### Лемматизация

In [6]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text).lower()
    text = text.split() 
    return ' '.join(text)

In [7]:
#m = Mystem()
m = WordNetLemmatizer() 
def lemmatize(text):
    text = clear_text(text)
    tokens = nltk.word_tokenize(text)
    return  ' '.join([m.lemmatize(token) for token in tokens])

In [8]:
train.loc[:]['text'] = train['text'].apply(lemmatize)

In [9]:
train.head()

,text,toxic
45800,i have offered a well but he must do my map fi...,0
94209,your biased orthodox view have no more merit t...,0
135210,thank you very much for your quick respond and...,0
89158,i moved this page when i did i found another l...,0
61233,gray powell article nominated for deletion nom...,0


In [10]:
test.loc[:]['text'] = test['text'].apply(lemmatize)

In [11]:
test.head()

,text,toxic
146790,ahh shut the fuck up you douchebag sand nigger...,1
2941,reply there is no such thing a texas commerce ...,0
115087,reply hey you could at least mention jasenovac...,0
48830,thats fine there is no deadline chi,0
136034,dyk nomination of mustarabim hello your submis...,0


### Создание признаков для обучения 

In [12]:
stopwords = set(nltk_stopwords.words('english'))

In [13]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
count_tf_idf.fit(train['text'])

TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [14]:
features = count_tf_idf.transform(train['text'])

In [15]:
train_x = features

In [16]:
train_y = train['toxic']

## Обучение

In [17]:
grid_report = pd.DataFrame(columns=['model', 'f1', 'params'])

In [18]:
f1 = make_scorer(f1_score)

In [19]:
def my_GridSearchCV(mode, parametrs, train_x, train_y):
    grid = GridSearchCV(model(), parametrs, cv=5, scoring=f1)
    grid.fit(train_x, train_y)
    
    row = [model.__name__, grid.best_score_, grid.best_params_]
    return pd.DataFrame([row], columns=['model', 'f1', 'params'])

### Простой классификатор   

In [20]:
parametrs = { 'strategy' : ('stratified', 'most_frequent', 'prior', 'uniform')}

In [21]:
model = DummyClassifier

In [22]:
result = my_GridSearchCV(model, parametrs, train_x, train_y)

In [23]:
grid_report = grid_report.append(result)

In [24]:
grid_report

,model,f1,params
0,DummyClassifier,0.167899,{'strategy': 'uniform'}


### Решающие дерево

In [25]:
parametrs = {'max_depth' : range(10, 50, 20)}

In [26]:
model = DecisionTreeClassifier

In [27]:
result = my_GridSearchCV(model, parametrs,train_x,train_y)

In [28]:
grid_report = grid_report.append(result)

In [29]:
grid_report

,model,f1,params
0,DummyClassifier,0.167899,{'strategy': 'uniform'}
0,DecisionTreeClassifier,0.673936,{'max_depth': 30}


### Логистическая регрессия

In [30]:
parametrs = { 'C': [1, 5, 10, 15],
             'random_state':[12345]}

In [31]:
model = LogisticRegression

In [32]:
result = my_GridSearchCV(model, parametrs,train_x,train_y)

In [33]:
grid_report = grid_report.append(result)

In [34]:
grid_report

,model,f1,params
0,DummyClassifier,0.167899,{'strategy': 'uniform'}
0,DecisionTreeClassifier,0.673936,{'max_depth': 30}
0,LogisticRegression,0.762924,"{'C': 15, 'random_state': 12345}"


## Тестирование 

In [35]:
grid_report.reset_index(drop=True, inplace=True)

In [36]:
grid_report.sort_values('f1', ascending=False)

,model,f1,params
2,LogisticRegression,0.762924,"{'C': 15, 'random_state': 12345}"
1,DecisionTreeClassifier,0.673936,{'max_depth': 30}
0,DummyClassifier,0.167899,{'strategy': 'uniform'}


### Подготовка признаков для тестирования

In [37]:
test_features = count_tf_idf.transform(test['text'])

### Проверка на адекватность 

#### Простой классификатор  

In [38]:
dummy_model = DummyClassifier(**grid_report.loc[0,'params'])

In [39]:
dummy_model.fit(train_x,train_y)

DummyClassifier(strategy='uniform')

In [40]:
predics = dummy_model.predict(test_features)

In [41]:
f1_score(test['toxic'],predics)

0.1724137931034483

#### Логистическая регрессия

In [42]:
reg_model = LogisticRegression(**grid_report.loc[2,'params'])

In [43]:
reg_model.fit(train_x,train_y)

LogisticRegression(C=15, random_state=12345)

In [44]:
predics = reg_model.predict(test_features)

In [45]:
f1_score(test['toxic'],predics)

0.7784206943498979

## Выводы

В ходе данного проекта была разработана модель для предсказания токсичных комментариев. Для ее создание данные были очищены и преобразованы из текста в числовые признаки с помощью TF-IDF преобразования. В исследовании максимальную точность показала логистическая регрессия. Ее F1 метрика составляет 0.7784206943498979.